## GRU model testing
OK, we have done carry forward control, we have done simple linear models, we have done ARIMA - last thing to try before going on a data hunt in true neural network based models. We could try some regression with vanilla dense layers or something, but I think we should cut to the chase and go straight to a GRU based network. Let's get a minimal example set up to get an idea of what we are working with and then set up a more rigorous hyperparameter optimization with crossvalidation/bootstrapping.

In [ ]:
# Add notebook parent dir to path so we can import from functions/
import sys
sys.path.append('..')

# Import notebooks specific helper functions
import functions.notebook_helper_functions.notebook15 as funcs

# Number of counties to include in our testing
num_counties = 100

# Load parsed data - note: 37 is the max block size because we lost
# the first rows of each timecourse to differencing
block_size = 37